In [ ]:
import os

import pandas as pd

In [ ]:
def load_discharge_table(path: str) -> pd.DataFrame:
    """
    Load discharge table with required columns:
    ['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq', 'charttime', 'storetime']
    """
    df = pd.read_csv(path, parse_dates=['charttime', 'storetime'])
    df = df.sort_values(['subject_id', 'storetime']).reset_index(drop=True)
    return df


def build_discharge_windows(discharge_df: pd.DataFrame) -> pd.DataFrame:
    """
    Given discharge table, create a dataframe of time windows per subject.
    Each window represents the time between consecutive storetime values.
    """
    discharge_df = discharge_df.copy()
    discharge_df['window_start'] = discharge_df.groupby('subject_id')['storetime'].shift(1)
    discharge_df['window_end'] = discharge_df['storetime']

    # The first discharge has no previous storetime, so window_start can be NaT or -inf
    discharge_df['window_start'] = discharge_df['window_start'].fillna(pd.Timestamp.min)
    return discharge_df[['subject_id', 'hadm_id', 'window_start', 'window_end']]


def extract_rows_in_windows(data_df: pd.DataFrame,
                            discharge_windows: pd.DataFrame,
                            time_col: str) -> pd.DataFrame:
    """
    Generic function to filter rows from data_df that fall within discharge windows.
    Arguments:
        data_df: dataframe with at least ['subject_id', time_col]
        discharge_windows: dataframe returned from build_discharge_windows
        time_col: name of timestamp column in data_df (must be datetime)
    Returns:
        DataFrame of rows joined to their corresponding discharge window
    """
    data_df = data_df.copy()
    data_df[time_col] = pd.to_datetime(data_df[time_col])

    results = []
    for subj, subj_windows in discharge_windows.groupby('subject_id'):
        subj_data = data_df[data_df['subject_id'] == subj]
        if subj_data.empty:
            continue

        for _, row in subj_windows.iterrows():
            mask = (subj_data[time_col] > row['window_start']) & (subj_data[time_col] <= row['window_end'])
            matched = subj_data.loc[mask].copy()
            if not matched.empty:
                matched['hadm_id_window'] = row['hadm_id']
                matched['window_start'] = row['window_start']
                matched['window_end'] = row['window_end']
                results.append(matched)

    if results:
        return pd.concat(results, ignore_index=True)
    return pd.DataFrame(columns=list(data_df.columns) + ['hadm_id_window', 'window_start', 'window_end'])


def concat_medications_per_discharge(filtered_df: pd.DataFrame) -> pd.DataFrame:
    """
    Given pharmacy rows already filtered into discharge windows, 
    return one row per discharge (hadm_id) with all medications concatenated.
    Assumes filtered_df has at least ['subject_id', 'hadm_id_window', 'medication'] columns.
    """
    # Keep only relevant columns
    cols = ['subject_id', 'hadm_id_window', 'medication']
    filtered_df = filtered_df[cols].copy()

    # Concatenate all medications per discharge
    grouped = (
        filtered_df
        .groupby(['subject_id', 'hadm_id_window'], dropna=False)
        .agg({'medication': lambda x: ', '.join(x.astype(str).unique())})
        .reset_index()
    )

    # Rename hadm_id_window back to hadm_id
    grouped = grouped.rename(columns={'hadm_id_window': 'hadm_id'})

    return grouped



In [ ]:
# Load discharge data
discharge_df = load_discharge_table("data_samples/notes/discharge.csv")

# Build discharge windows
windows = build_discharge_windows(discharge_df)

# Load another table (e.g. pharmacy.csv)
pharmacy_df = pd.read_csv("data_samples/hosp/pharmacy.csv", parse_dates=['starttime'])

# Extract pharmacy rows that fall into each discharge window
pharmacy_in_windows = extract_rows_in_windows(pharmacy_df, windows, time_col='starttime')

# Optionally group by discharge
med_concat_per_discharge = concat_medications_per_discharge(pharmacy_in_windows)

med_concat_per_discharge


In [ ]:
discharge_df = load_discharge_table("data_samples/notes/discharge.csv")

# Build discharge windows
build_discharge_windows(discharge_df)

## Descrever, como foi feitas as janelas - como foi selecinadas os periodos e pq
## 

In [ ]:
import pandas as pd
import os
HOSP = '../data_samples/hosp/'
ICU = '../data_samples/icu/'

info

In [ ]:
# Identity
# Age & Gender
# Race & Language: Important for social history and potential care barriers.
# Admission Type & Location: (e.g., "Urgent admission via the Emergency Room"). How critical the situation was.
# Baselines BMI/Weight...
patients = pd.read_csv(HOSP+'patients.csv')
admissions = pd.read_csv(HOSP+'admissions.csv')
omr = pd.read_csv(HOSP+'omr.csv')

In [ ]:


# 1. Standardize Timestamps
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
patients['anchor_year'] = patients['anchor_year'].astype(int)

# 2. Merge Admissions and Patients
# We bring in gender and the age anchors
identity_df = admissions.merge(
    patients[['subject_id', 'gender', 'anchor_age', 'anchor_year', 'dod']], 
    on='subject_id', 
    how='left'
)

# 3. Calculate Age at Admission
# Age = anchor_age + (Year of Admission - anchor_year)
identity_df['admission_year'] = identity_df['admittime'].dt.year
identity_df['age_at_admission'] = (identity_df['anchor_age'] + (identity_df['admission_year'] - identity_df['anchor_year']))

# 4. Integrate OMR (Baseline Vitals)
# Height, Weight, BMI, Blood pressure
baseline_omr = omr[omr['result_name'].isin(['Weight (Lbs)', 'Height (Inches)', 'BMI (kg/m2)', 'Blood Pressure'])].copy()
baseline_omr = baseline_omr.sort_values('chartdate').groupby(['subject_id', 'result_name']).head(1)

# Pivot so each measurement is its own column
omr_pivot = baseline_omr.pivot(index='subject_id', columns='result_name', values='result_value').reset_index()

# Merge OMR data into main identity frame
identity_df = identity_df.merge(omr_pivot, on='subject_id', how='left')

# 5. Set the Anchor Timestamp
identity_df['timestamp'] = identity_df['admittime']

# 6. Cleanup and Selection
# Rename OMR columns to be more regex-friendly (remove spaces/units if preferred)
identity_df = identity_df.rename(columns={
    'Weight (Lbs)': 'weight_baseline',
    'Height (Inches)': 'height_baseline',
    'BMI (kg/m2)': 'bmi_baseline',
    'Blood Pressure': 'blood_pressure_baseline',
})

relevant_cols = [
    'subject_id', 'hadm_id', 'timestamp', 'age_at_admission', 'gender', 'race',
    'admission_type', 'admission_location', 'insurance', 'marital_status',
    'weight_baseline', 'height_baseline', 'bmi_baseline', 'blood_pressure_baseline', 'dod'
]

identity_df = identity_df[relevant_cols]

In [ ]:
# Transfers, Services, and Admissions are grouped as Logistics, because a hospital stay is defined by movement through locations
# and care teams; Transfers describe where the patient was (ED, ICU, ward), while Services identify which clinical team was responsible,
# forming the structural backbone of the hospitalization narrative. Finally, all time-related fields are unified under a single
# timestamp, since different tables record events using different time columns; standardizing these enables all events to be ordered
# chronologically, transforming disconnected facts into a coherent, step-by-step timeline of the patient's hospital course.

transfers = pd.read_csv(HOSP+'transfers.csv')
services = pd.read_csv(HOSP+'services.csv')

# 1. Standardize Timestamps
transfers['intime'] = pd.to_datetime(transfers['intime'])
transfers['outime'] = pd.to_datetime(transfers['outtime'])
services['transfertime'] = pd.to_datetime(services['transfertime'])

# 2. Fix HADM_ID types to avoid MergeError
# We drop NaNs first because you can't join on an empty ID
transfers = transfers.dropna(subset=['hadm_id'])
services = services.dropna(subset=['hadm_id'])

# Force to int64
transfers['hadm_id'] = transfers['hadm_id'].astype('int64')
services['hadm_id'] = services['hadm_id'].astype('int64')
admissions['hadm_id'] = admissions['hadm_id'].astype('int64')

# 3. Enrich Transfers with Admission/Discharge info
logistics_df = transfers.merge(
    admissions[['hadm_id', 'admission_location', 'discharge_location']], 
    on='hadm_id', 
    how='left'
)

# 4. Join Services (The "As-Of" Join)
services = services.sort_values('transfertime')
logistics_df = logistics_df.sort_values('intime')

logistics_df = pd.merge_asof(
    logistics_df,
    services[['hadm_id', 'curr_service', 'transfertime']],
    left_on='intime',
    right_on='transfertime',
    by='hadm_id',
    direction='backward'
)

# 5. Filter for real ward movements and calculate duration
logistics_df = logistics_df.dropna(subset=['careunit'])
logistics_df['stay_duration_hours'] = (logistics_df['outime'] - logistics_df['intime']).dt.total_seconds() / 3600

# 6. Set the Anchor Timestamp
logistics_df['timestamp'] = logistics_df['intime']

relevant_cols = [
    'subject_id', 'hadm_id', 'timestamp', 'eventtype', 'careunit', 
    'curr_service', 'intime', 'outime', 'stay_duration_hours',
    'admission_location', 'discharge_location'
]

logistics_df = logistics_df[relevant_cols]

In [ ]:
d_items = pd.read_csv(ICU+'d_items.csv')
chartevents = pd.read_csv(ICU+'chartevents.csv')

# 1. Filter d_items for common Vital Signs to keep the data manageable
# In MIMIC-IV, these are the most common ICU vital codes
vital_itemids = [220045, 220179, 220180, 220210, 223761] # HR, SysBP, DiasBP, RR, Temp
vitals_dict = d_items[d_items['itemid'].isin(vital_itemids)]

# 2. Join ICU Vitals
icu_vitals = chartevents.merge(vitals_dict[['itemid', 'label']], on='itemid', how='inner')
icu_vitals['timestamp'] = pd.to_datetime(icu_vitals['charttime'])

# 3. Join Ward Vitals (from OMR)
ward_vitals = omr[omr['result_name'].str.contains('Blood Pressure', na=False)].copy()
ward_vitals['timestamp'] = pd.to_datetime(ward_vitals['chartdate'])

# Merge with admissions to get the hadm_id for each OMR measurement
# Note: Since OMR doesn't have hadm_id, we link via subject_id and time
ward_vitals = ward_vitals.merge(admissions[['subject_id', 'hadm_id', 'admittime', 'dischtime']], on='subject_id', how='left')

# Filter to keep only OMR records that happened DURING the admission
ward_vitals['admittime'] = pd.to_datetime(ward_vitals['admittime'])
ward_vitals['dischtime'] = pd.to_datetime(ward_vitals['dischtime'])

mask = (ward_vitals['timestamp'] >= ward_vitals['admittime'].dt.normalize()) & \
       (ward_vitals['timestamp'] <= ward_vitals['dischtime'].dt.normalize())
ward_vitals = ward_vitals[mask]

# Now rename and select columns
ward_vitals = ward_vitals.rename(columns={'result_name': 'label', 'result_value': 'valuenum'})

# 4. Combine into a "Master Vitals" table
# Now ward_vitals has 'hadm_id', so the concatenation won't crash
monitoring_df = pd.concat([
    icu_vitals[['subject_id', 'hadm_id', 'timestamp', 'label', 'valuenum', 'valueuom']],
    ward_vitals[['subject_id', 'hadm_id', 'timestamp', 'label', 'valuenum']]
], axis=0)

monitoring_df = monitoring_df.sort_values('timestamp')

In [ ]:
labevents = pd.read_csv(HOSP+'labevents.csv')
d_labitems = pd.read_csv(HOSP+'d_labitems.csv')
microbiology = pd.read_csv(HOSP+'microbiologyevents.csv')

# 1. Standardize Lab Events
labevents['timestamp'] = pd.to_datetime(labevents['charttime'])

# Merge with dictionary to get names (Labels) and the specimen type (Fluid)
labs_df = labevents.merge(
    d_labitems[['itemid', 'label', 'fluid', 'category']], 
    on='itemid', 
    how='left'
)

# Select key columns for the summary
# 'flag' is vital here: it tells us if a result was 'abnormal'
labs_final = labs_df[[
    'subject_id', 'hadm_id', 'timestamp', 'label', 'valuenum', 
    'valueuom', 'flag', 'fluid'
]].copy()
labs_final['type'] = 'LAB'

# 2. Process Microbiology
microbiology['timestamp'] = pd.to_datetime(microbiology['charttime'])

# We focus on the specimen (what was tested) and the organism (what was found)
# If org_name is NaN, it usually means "No growth"
micro_final = microbiology[[
    'subject_id', 'hadm_id', 'timestamp', 'spec_type_desc', 
    'org_name', 'ab_name', 'interpretation'
]].copy()

micro_final = micro_final.rename(columns={'spec_type_desc': 'label'})
micro_final['type'] = 'MICRO'

# 3. Combine into the Investigations Table
investigations_df = pd.concat([labs_final, micro_final], axis=0)

investigations_df = investigations_df.sort_values('timestamp')

In [ ]:
investigations_df

In [ ]:
# Procedures
procedureevents = pd.read_csv(ICU+'procedureevents.csv')
d_items = pd.read_csv(ICU+'d_items.csv')

d_icd_procedures = pd.read_csv(HOSP+'d_icd_procedures.csv')
procedures_icd = pd.read_csv(HOSP+'procedures_icd.csv')

In [ ]:

# 1. ICU Bedside Procedures
procedureevents['timestamp'] = pd.to_datetime(procedureevents['starttime'])
procedureevents['endtime'] = pd.to_datetime(procedureevents['endtime'])

icu_procs = procedureevents.merge(d_items[['itemid', 'label']], on='itemid', how='left')
icu_procs['proc_type'] = 'ICU_BEDSIDE'

# 2. Billed Hospital Procedures (The "Translation" Fix)
# Force versions to int and codes to padded strings
for df in [procedures_icd, d_icd_procedures]:
    df['icd_version'] = df['icd_version'].astype(int)
    df['icd_code'] = df['icd_code'].astype(str).str.strip()
    # Pad ICD-9 codes to 4 digits (e.g., '66' -> '0066')
    mask_v9 = df['icd_version'] == 9
    df.loc[mask_v9, 'icd_code'] = df.loc[mask_v9, 'icd_code'].str.zfill(4)

billed_procs = procedures_icd.merge(
    d_icd_procedures[['icd_code', 'icd_version', 'long_title']], 
    on=['icd_code', 'icd_version'], 
    how='left'
)

# Billed procedures only have a 'chartdate'. We anchor them to 'admittime' 
# so they appear in the stay window, but label them as billed events.
billed_procs = billed_procs.merge(admissions[['hadm_id', 'admittime']], on='hadm_id', how='left')
billed_procs['timestamp'] = pd.to_datetime(billed_procs['admittime'])
billed_procs = billed_procs.rename(columns={'long_title': 'label'})
billed_procs['proc_type'] = 'BILLED_SURGICAL'

# 3. Combine into Interventions Table
interventions_df = pd.concat([
    icu_procs[['subject_id', 'hadm_id', 'timestamp', 'label', 'proc_type', 'endtime']],
    billed_procs[['subject_id', 'hadm_id', 'timestamp', 'label', 'proc_type']]
], axis=0)

interventions_df.sort_values('timestamp')

In [ ]:
inputevents = pd.read_csv(ICU+'inputevents.csv')
d_items = pd.read_csv(ICU+'d_items.csv')

prescriptions = pd.read_csv(HOSP+'prescriptions.csv')

In [ ]:
# 1. ICU Continuous Inputs (IVs and Drips)
inputevents['timestamp'] = pd.to_datetime(inputevents['starttime'])

# Join with d_items to get drug names
icu_inputs = inputevents.merge(d_items[['itemid', 'label']], on='itemid', how='left')

# We keep amount and rate to describe the intensity of the treatment
icu_inputs = icu_inputs[['subject_id', 'hadm_id', 'timestamp', 'label', 'amount', 'amountuom', 'rate', 'rateuom']]
icu_inputs['input_type'] = 'ICU_INPUT'

# 2. General Hospital Prescriptions
prescriptions['timestamp'] = pd.to_datetime(prescriptions['starttime'])

# We select the drug name, dose, and route (e.g., PO for mouth, IV for vein)
ward_inputs = prescriptions[[
    'subject_id', 'hadm_id', 'timestamp', 'drug', 'dose_val_rx', 'dose_unit_rx', 'route'
]].copy()

ward_inputs = ward_inputs.rename(columns={'drug': 'label'})
ward_inputs['input_type'] = 'WARD_PRESCRIPTION'

# 3. Combine into a Master Meds Table
meds_df = pd.concat([icu_inputs, ward_inputs], axis=0)
    
meds_df.sort_values('timestamp')

In [ ]:
diagnoses_icd = pd.read_csv(HOSP+'diagnoses_icd.csv')
d_icd_diagnoses = pd.read_csv(HOSP+'d_icd_diagnoses.csv')

In [ ]:

# 1. Clean and Pad ICD Codes (Same logic as Procedures)
for df in [diagnoses_icd, d_icd_diagnoses]:
    df['icd_version'] = df['icd_version'].astype(int)
    df['icd_code'] = df['icd_code'].astype(str).str.strip()
    # Pad ICD-9 codes to 3-5 digits depending on the code type if necessary
    # Usually, a simple zfill handles the majority of join misses
    mask_v9 = df['icd_version'] == 9
    df.loc[mask_v9, 'icd_code'] = df.loc[mask_v9, 'icd_code'].str.zfill(3)

# 2. Join with Dictionary
outcomes_df = diagnoses_icd.merge(
    d_icd_diagnoses[['icd_code', 'icd_version', 'long_title']], 
    on=['icd_code', 'icd_version'], 
    how='left'
)

# 3. Anchor to Discharge Time
outcomes_df = outcomes_df.merge(
    admissions[['hadm_id', 'dischtime']], 
    on='hadm_id', 
    how='left'
)

outcomes_df['timestamp'] = pd.to_datetime(outcomes_df['dischtime'])
outcomes_df = outcomes_df.rename(columns={'long_title': 'diagnosis_label'})

# 'seq_num' tells us the priority (1 is the primary diagnosis)
outcomes_df[['subject_id', 'hadm_id', 'timestamp', 'diagnosis_label', 'seq_num']].sort_values('seq_num')

In [ ]:
def get_diagnosis_summary(outcomes_df, target_hadm_id):
    # Filter for the specific admission
    stay_outcomes = outcomes_df[outcomes_df['hadm_id'] == target_hadm_id].sort_values('seq_num')
    
    # Get the list of diagnosis labels
    conditions = stay_outcomes['diagnosis_label'].tolist()
    
    if not conditions:
        return "No recorded diagnoses found for this stay."
    
    # Construct the clean string
    condition_list = ", ".join(conditions)
    summary_str = f"During the stay, the patient was diagnosed with the following conditions: {condition_list}."
    
    return summary_str

In [ ]:
def context_builder_v2(window_cuts):
    """
    Inputs: window_cuts -> dict of DataFrames (identity, logistics, monitoring, etc.)
    already sliced for one specific patient and one specific time window.
    Output: A single stitched string.
    """
    paragraphs = []

    # 1. Identity (The Intro)
    # We assume identity is passed in the dict; usually contains 1 row per admission
    if 'identity' in window_cuts and not window_cuts['identity'].empty:
        id_df = window_cuts['identity']
        row = id_df.iloc[0] # Take the most recent/relevant identity row
        paragraphs.append(
            f"Patient is a {row.age_at_admission} year old {row.gender} ({row.race}) "
            f"admitted via {row.admission_location}."
        )

    # 2. Logistics (The Journey)
    if 'logistics' in window_cuts and not window_cuts['logistics'].empty:
        log = window_cuts['logistics'].sort_values('timestamp')
        units = " -> ".join(log['careunit'].unique())
        paragraphs.append(f"Clinical course involved the following units: {units}.")

    # 3. Monitoring (The Vitals)
    if 'monitoring' in window_cuts and not window_cuts['monitoring'].empty:
        mon = window_cuts['monitoring'].sort_values('timestamp')
        # Get the latest reading for each unique vital sign in this window
        latest_vitals = mon.groupby('label').tail(1)
        v_list = [f"{r.label}: {r.valuenum}{getattr(r, 'valueuom', '')}" for r in latest_vitals.itertuples()]
        paragraphs.append(f"Most recent vital signs: {', '.join(v_list)}.")

    # 4. Investigations (The Labs)
    if 'investigations' in window_cuts and not window_cuts['investigations'].empty:
        inv = window_cuts['investigations']
        # Focus on abnormal results to keep the text high-signal
        abnormal = inv[inv['flag'] == 'abnormal'].sort_values('timestamp')
        if not abnormal.empty:
            l_list = [f"{r.label} ({r.valuenum})" for r in abnormal.itertuples()]
            paragraphs.append(f"Significant lab abnormalities identified: {', '.join(l_list)}.")

    # 5. Inputs (The Meds)
    if 'inputs' in window_cuts and not window_cuts['inputs'].empty:
        meds = window_cuts['inputs']
        med_names = meds['label'].unique()
        paragraphs.append(f"Treatments administered during this window included: {', '.join(med_names)}.")

    # 6. Interventions (The Actions)
    if 'interventions' in window_cuts and not window_cuts['interventions'].empty:
        procs = window_cuts['interventions']['label'].unique()
        paragraphs.append(f"Procedures performed: {', '.join(procs)}.")

    # 7. Conclusion (The Diagnoses)
    if 'conclusion' in window_cuts and not window_cuts['conclusion'].empty:
        diag = window_cuts['conclusion'].sort_values('seq_num')
        diag_list = diag['diagnosis_label'].tolist()
        paragraphs.append(f"During the stay, the patient was diagnosed with the following conditions: {', '.join(diag_list)}.")

    # Stitch all paragraphs into one clean block
    return " ".join(paragraphs)